# Demonstrate loading and using ERNIE4us

## Demo loading the ERNIE2 model and preparing inputs

In [ ]:
import dataclasses
from ernie4us import *

In [ ]:
tf.__version__

In [ ]:
ernie_input_builder, ernie_model = load_ernie_model(ERNIE_BASE_EN, "./model_artifacts")

## Demo ERNIE tensorflow 2 usages

In [ ]:
ernie_model.max_seq_length

In [ ]:
text_a = "ERNIE for the rest of us"
text_b = "thank you ERNIE"

record = ernie_input_builder.build(text_a, text_b,task_id=0)
for key, item in dataclasses.asdict(record).items():
    print(f'{key}', item.shape, '=>', np.reshape(item, (ernie_model.max_seq_length,))[:20])

In [ ]:
type(record), [(v.shape, v.dtype) for v in record.as_tuple()]

In [ ]:
outputs = ernie_model(record.as_tuple())
outputs = Ernie2Output(sequence_features=outputs[0], classification_features=outputs[1])
outputs.classification_features.shape, outputs.sequence_features.shape

In [ ]:
import time

ernie_inputs = record.as_tuple()
st = time.time()
n_times = 20
for _ in range(n_times):
    ernie_model(ernie_inputs)
dt = time.time() - st
print(f"finished in {dt}s. avg {dt / n_times}s/request")

In [ ]:
num_class = 3
with tf.name_scope("classifier"):
    dense_layer = tf.keras.layers.Dense(num_class)
    
    def classifier(ernie_inputs):
        _, classification_features = ernie_model(ernie_inputs)
        return dense_layer(classification_features)

In [ ]:
outputs = classifier(record.as_tuple())
outputs

## Demonstrate Keras model

In [ ]:
ernie_inputs, ernie_outputs = ernie_model.create_keras_invocation()
classifier_layer = tf.keras.layers.Dense(num_class)
logits = classifier_layer(ernie_outputs.classification_features)
keras_model = tf.keras.Model(inputs=ernie_inputs.as_tuple(), outputs=[logits])
keras_model.compile()
keras_model.summary()

In [ ]:
tf.saved_model.save(keras_model, '/tmp/ernie_classifier')

In [ ]:
!ls -lhR /tmp/ernie_classifier